# Le but de ce notebook est d'identifier des caractéristiques permettant de différencier des textes écrits par Molière et Corneille

# Installation des modules nécessaires

In [ ]:
!pip install unidecode

# PRIVE: Hyperparamétres fixés par MathAData

In [ ]:
# nombre de caractéres dans chaque paragraphe dont on veut identifier l'auteur
# 1000 caractères correspond à environ une page de texte (environ 180 mots)
min_paragraph_length = 1000

# on divise les données d'entraînement en 90% en train et 10% en validation
# une oeuvre d'un auteur donné doit être entièrement soit en train soit en validation
percentage_in_train = 0.9

# on ignore les majuscules / minuscules dans les mots: Monsieur == monsieur
use_lowercase = True

# on ignore les accents:  être == etre
use_diacritics = True

# nombre de mots signatures chez chaque auteur
# on ne garde que les 'most_common_normalized_words_count' mots les plus courants chez les auteurs
most_common_normalized_words_count = 100

# Mots vides. 
# Ils sont ignorés par l'outil car très communs à la fois chez Molière et chez Corneille
stop_words = set(["de","et","que","je","a","la","le","ne","ce","il","pour","un","qui","me","est","mais","des","moi","votre","qu'il","lui","du","fait","par","se","au","cette","sur","j'ai","avec","tous","vos","ces","n'est","peu","peut","quelque","dont","quoi","aux","donc","d'une","s'il","notre","sais","donne","vois","m'en","cet","autre","puis","assez","quel","veut","va","ils","doit","ont","vu", "en", "les", "vous", "mon","pas","si","plus", "tout", "nous", "ma", "sans", "ou", "c'est", "bien", "dans","une", "son","tu", "point", "mais", "mes", "d'un", "elle", "ses", "meme", "comme", "te", "sa", "ton", "ta", "sganarelle","jourdain","mascarille", "dom"])



# PRIVE: Méthodes utilisés dans le Notebook

In [ ]:
import functools
from typing import List,Set,Tuple,Dict
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter,MaxNLocator     
import numpy as np
from scipy.interpolate import make_interp_spline
import os
import pathlib
import re
import os
import random
import math
import pandas as pd
import numpy as np


# le répertoire de travail
directory = os.path.abspath('')
random.seed(42)

# retourne tous les fichiers *.txt présents dans le repertoire 'path'
def all_txt_files_in_directory(path: str):
    return [os.path.join(path,f) for f in os.listdir(path) if os.path.isfile(os.path.join(path, f)) and f.endswith('.txt')]

# infique si la ligne 'ligne' est une ligne valide ou si elle doit être ignorée (par exemple si elle vide)
def is_valid_line(line: str) -> bool:
    if line.startswith('Scène ') or line.startswith('Acte '):
        return False
    if len(line)<10:
        return False
    return True

def split_book_into_paragraphs(path: str) -> List[str]:
    result = []
    with open(path, encoding='latin1') as file:
        current_paragraph = ""
        for line in file:
            if is_valid_line(line):
                if current_paragraph :
                    current_paragraph += "\n"
                current_paragraph += line.rstrip()
                if len(current_paragraph) >= min_paragraph_length:
                    result.append(current_paragraph) 
                    current_paragraph = ""
    if len(current_paragraph) >= min_paragraph_length or (current_paragraph and len(result) == 0):
        result.append(current_paragraph) 
    return result

def load_all_books(path: str) -> Dict[str,List[str]]:
    book_to_paragraphs = dict()
    for book_path in all_txt_files_in_directory(path):
        book_to_paragraphs[pathlib.Path(book_path).stem] = split_book_into_paragraphs(book_path)
    return book_to_paragraphs

# pour supprimer les accents
@functools.lru_cache(maxsize=None)
def remove_diacritics(word: str) -> str:
    import unidecode  
    return unidecode.unidecode(word)

# mots en minuscules
@functools.lru_cache(maxsize=None)
def to_lowercase(word: str) -> str:
    return word.lower()

@functools.lru_cache(maxsize=None)
def compute_normalized_word(word: str) -> str:
    if use_lowercase:
        word = to_lowercase(word)
    if use_diacritics:
        word = remove_diacritics(word)
    return word

def paragraph_count(book_to_paragraphs: Dict[str, List[str]]) -> int:
    if not book_to_paragraphs:
        return 0
    return sum([len(c) for c in book_to_paragraphs.values()])

def split_text(text:str) -> List[str]:
    return re.findall(r"\b[\w'^\d]+\b", text.rstrip())
            
def word_count(book_to_paragraphs: Dict[str, List[str]]) -> int:
    result = 0
    for paragraph in all_paragraphs(book_to_paragraphs):
        result += len(split_text(paragraph.rstrip()))
    return result

def all_paragraphs(book_to_paragraphs: Dict[str, List[str]]) -> List[str]:
    result = []
    for p in book_to_paragraphs.values():
        result.extend(p)
    return result

# reduce the dataset so that it contains exactly 'target_count' paragraphs
def reduce_to_paragraph_count(book_to_paragraphs: dict, target_count: int ) -> int:
    current_count = paragraph_count(book_to_paragraphs)
    if current_count<target_count:
        raise Exception(f'current_count {current_count} < target_count {target_count}')
    to_remove = current_count-  target_count
    result = dict()
    for book, paragraphs in sorted(book_to_paragraphs.items(), key =lambda x : len(x[1])):
        if len(paragraphs)<=to_remove:
            to_remove-=len(paragraphs)
            continue
        result[book] = paragraphs[:len(paragraphs)-to_remove]
        to_remove = 0
    return result

def compute_normalized_words_to_stats_without_stop_words(paragraphs: List[str]) -> Dict[str, Tuple[int,Dict[str,int]] ]:
    normalized_words_to_stats = dict()
    for paragraph in paragraphs:
        words = split_text(paragraph)
        for original_word in words:
            normalized_word = compute_normalized_word(original_word)
            if normalized_word in stop_words:
                continue
            if normalized_word not in normalized_words_to_stats:
                normalized_words_to_stats[normalized_word] = (0, dict())
            count,original_word_count = normalized_words_to_stats[normalized_word]
            if original_word not in original_word_count:
                original_word_count[original_word] = 1
            else:
                original_word_count[original_word] += 1
            normalized_words_to_stats[normalized_word] = (count+1,original_word_count)
    return normalized_words_to_stats

def split_train_validation_single_author(book_to_paragraphs: dict, percentage_in_train:float) :
    books = list(book_to_paragraphs.keys())
    train = dict()
    validation = dict()
    for book, paragraphs in book_to_paragraphs.items():
        paragraphs_count = len(paragraphs)
        percentage_in_train_if_adding_to_train = (paragraph_count(train)+len(paragraphs))/max(paragraph_count(train)+paragraph_count(validation)+len(paragraphs),1)
        percentage_in_train_if_adding_to_validation = paragraph_count(train)/max(paragraph_count(train)+paragraph_count(validation)+len(paragraphs),1)
        if abs(percentage_in_train_if_adding_to_train-percentage_in_train)<abs(percentage_in_train_if_adding_to_validation-percentage_in_train):
            train[book] = paragraphs
        else:
            validation[book] = paragraphs
    return train, validation

def split_train_validation_all_authors(book_to_paragraphs_author1: dict, book_to_paragraphs_author2: dict, percentage_in_train:float) :
    train_author1,validation_author1 = split_train_validation_single_author(book_to_paragraphs_author1, percentage_in_train)
    train_author2,validation_author2 = split_train_validation_single_author(book_to_paragraphs_author2, percentage_in_train)

    target_length_validation = min(paragraph_count(validation_author1),paragraph_count(validation_author2))            
    validation_author1 = reduce_to_paragraph_count(validation_author1, target_length_validation)
    validation_author2 = reduce_to_paragraph_count(validation_author2, target_length_validation)

    target_length_train = min(paragraph_count(train_author1),paragraph_count(train_author2))            
    train_author1 = reduce_to_paragraph_count(train_author1, target_length_train)
    train_author2 = reduce_to_paragraph_count(train_author2, target_length_train)

    proportion_in_train = target_length_train/(target_length_train+target_length_validation)
    if proportion_in_train>percentage_in_train:
        target_length_train = int( (percentage_in_train/(1-percentage_in_train)) *target_length_validation )
        train_author1 = reduce_to_paragraph_count(train_author1, target_length_train)
        train_author2 = reduce_to_paragraph_count(train_author2, target_length_train)
    else:
        target_length_validation = int( ((1-percentage_in_train)/percentage_in_train) *target_length_train )
        validation_author1 = reduce_to_paragraph_count(validation_author1, target_length_validation)
        validation_author2 = reduce_to_paragraph_count(validation_author2, target_length_validation)
    return train_author1,validation_author1,train_author2,validation_author2

def calcul_erreur(TP: int, TN: int, FP: int, FN: int):
    return 1-(TP+TN)/max(TP+TN+FP+FN,1)

def valeur_caracteristique(text:str, index_left:int, index_right:int) -> float:
    values = compute_words_count_in_text(text)
    return 100*sum(values[index_left: index_right+1])/sum(values)

def calcul_confusion_matrix_1_caracteristique(text_moliere: List[str], text_corneille: List[str], index_left:int, index_right:int, seuil_caracteristique:float) ->Tuple[int,int,int,int]:
    TP = 0 # y_true = Molière ,  y_pred = Molière
    TN = 0 # y_true = Corneille, y_pred = Corneille
    FN = 0 # y_true = Molière,   y_pred = Corneille
    FP = 0 # y_true = Corneille, y_pred = Molière 
    for t in text_moliere:
        if valeur_caracteristique(t, index_left, index_right) > seuil_caracteristique:
            TP += 1
        else:
            FN += 1
    for t in text_corneille:
        if valeur_caracteristique(t, index_left, index_right) > seuil_caracteristique:
            FP += 1
        else:
            TN += 1
    return (TP,TN,FP,FN)
        
    
def calcul_validation_erreur_moliere_vs_corneille_1_caracteristique(index_left:int, index_right:int, seuil_caracteristique:float) -> Tuple[float,float]:
    return calcul_erreur(*calcul_confusion_matrix_1_caracteristique(all_paragraphs(moliere_validation_dataset), all_paragraphs(corneille_validation_dataset), index_left, index_right, seuil_caracteristique))


@functools.lru_cache(maxsize=None)
def compute_words_count_in_text(text: str) -> List[int]:
    word_to_index = dict()
    for i in range (0, most_common_normalized_words_count):
        word_to_index[most_used_words[i]] = i
    res = [0] * most_common_normalized_words_count
        
    for original_word in split_text(text):
        normalized_word = compute_normalized_word(original_word)
        if normalized_word in word_to_index:
            res[word_to_index[normalized_word]] += 1
    return res


@functools.lru_cache(maxsize=None)
def get_values_mean_mode_interquartiles_indexes(text: str) -> Tuple[ List[int], float, float, float, float, float ]:
    values = compute_words_count_in_text(text)
    total = sum(values)
    index_Q1 = -1
    index_median_Q2 = -1
    index_Q3 = -1
    values_sum = 0
    values_sum_left = 0
    total_sum = 0
    index_mode = 0
    for i in range(0, len(values)):
        values_sum += values[i]
        if i<(len(values)/2):
            values_sum_left += values[i]
        if values[i] > values[index_mode]:
            index_mode = i            
        total_sum += (i+1)*values[i]
        if index_Q1 ==-1 and values_sum>=(0.25*total):
            index_Q1 = i
        if index_median_Q2 ==-1 and values_sum>=(0.5*total):
            index_median_Q2 = i
        if index_Q3 ==-1 and values_sum>=(0.75*total):
            index_Q3 = i
    return (values, total_sum/values_sum, 100-100*values_sum_left/values_sum, index_mode, index_Q1, index_median_Q2, index_Q3)

def affiche_erreur_1_caracteristique(index_left:int, index_right:int):
    seuil_caracteristiques = list(np.arange(0,1.00001,0.01))
    erreur_for_seuil_caracteristiques = []
    
    min_seuil_caracteristique = None
    min_validation_erreur = None
    
    for seuil_caracteristique in seuil_caracteristiques:
        validation_erreur = calcul_validation_erreur_moliere_vs_corneille_1_caracteristique(index_left, index_right, 100*seuil_caracteristique)
        erreur_for_seuil_caracteristiques.append(validation_erreur)
        if min_seuil_caracteristique is None or validation_erreur< min_validation_erreur:
            min_seuil_caracteristique = seuil_caracteristique
            min_validation_erreur = validation_erreur
        
    x_dense = np.linspace(min(seuil_caracteristiques), max(seuil_caracteristiques), 500)  # 500 points pour une courbe lisse
    spline = make_interp_spline(seuil_caracteristiques, erreur_for_seuil_caracteristiques)
    y_dense = spline(x_dense)

    plt.figure(figsize=(20, 10))
    plt.plot(x_dense, y_dense, label='Erreur', color='b')
    plt.scatter(seuil_caracteristiques, erreur_for_seuil_caracteristiques, color='r')
    plt.gca().tick_params(axis='y', which='major', labelsize=20) 
    plt.gca().xaxis.set_major_formatter(PercentFormatter(1, decimals=0))
    plt.gca().yaxis.set_major_formatter(PercentFormatter(1, decimals=0))
    plt.xticks(fontsize=20)
    plt.xlim(0, max(seuil_caracteristiques))
    plt.ylim(0, max(erreur_for_seuil_caracteristiques)+0.05)
    plt.xlabel(f"Seuil utilisé.\nSi le % de mots dans l'intervalle [{index_left},{index_right}] est supérieure à ce seuil,\nle texte sera attribué à Molière.", fontsize=20)
    plt.ylabel(f'Erreur pour distinguer des oeuvres de Molière et Corneille', fontsize=20)
    plt.title(f"Evolution de l'erreur en fonction du seuil choisi\nCaractéristique utilisée: % de mots dans l'intervalle [{index_left},{index_right}]", fontsize=20)
    #plt.legend()
    print(f'Min(error) = {min_validation_erreur} (seuil= {min_seuil_caracteristique})')
        
def display_vertical_line(plt, index, title:str, color:str) -> None:
    label = f'{title}: {index+1:.0f}'
    plt.axvline(index, color=color, linestyle='dashed', linewidth=1.5, label=label)
    plt.text(index, plt.ylim()[1] * 0.7, label, color=color, rotation=90, verticalalignment='center', fontsize=20)

def affiche_rapport_de_frequences(text:str, title:str, display_in_percentage: bool, display_left_rectangle:bool = False, display_right_rectangle:bool = False):
    values, index_mean, meanV2, index_mode, index_Q1, index_Q2, index_Q3 = get_values_mean_mode_interquartiles_indexes(text)
    plt.figure(figsize=(20, 5))
    if display_in_percentage:
        sum_elements = float(sum(map(abs, values)))
        values = [ v /sum_elements for v in values]
        
    plt.bar(range(most_common_normalized_words_count), values, color='blue', tick_label=most_used_unnormalized_word)
    # Add title and labels
    plt.title(title, fontsize=25)
    #plt.xlabel('Mot', fontsize=15)
    if display_in_percentage:
        plt.gca().yaxis.set_major_formatter(PercentFormatter(1))
        plt.ylabel(f"Fréquence d'occurences", fontsize=25)
    else:
        plt.gca().yaxis.set_major_locator(MaxNLocator(integer=True))
        plt.ylabel(f"Nombre d'occurences", fontsize=25)
    plt.gca().tick_params(axis='y', which='major', labelsize=15) 
    plt.xticks(rotation=90, fontsize=15)
    plt.xlim(-0.5, most_common_normalized_words_count - 0.5)
    plt.ylim(-1, 1.1*max(values))
    
    
    linewidth_rectangle = 1
    fontsize_rectangle = 12
    if display_left_rectangle: # Dessine un rectangle à gauche du diagramme de rapport de fréquences
        min_rect = 0
        max_rect = len(values)/2
        plt.gca().add_patch(plt.Rectangle((min_rect-0.5, 0), max_rect, max(values), edgecolor='red',facecolor='none', linewidth=linewidth_rectangle, hatch='//'))
        percentage_left = sum(values[0:len(values)//2])/sum(values)
        plt.text(min_rect+0.5*(max_rect-min_rect), 1.05*max(values) , f"{int(100*percentage_left)}% des mots à gauche du diagramme de rapport de fréquences (mots plus fréquents chez Molière)", color='red', fontsize=fontsize_rectangle, ha='center', va='center')
    if display_right_rectangle: # Dessine un rectangle à droite du diagramme de rapport de fréquences
        min_rect = len(values)/2
        max_rect = len(values)
        plt.gca().add_patch(plt.Rectangle((min_rect-0.5, 0), max_rect, max(values), edgecolor='red',facecolor='none', linewidth=linewidth_rectangle, hatch='//'))
        percentage_right = sum(values[len(values)//2:])/sum(values)
        plt.text(min_rect+0.5*(max_rect-min_rect), 1.05*max(values) , f"{int(100*percentage_right)}% des mots à droite du diagramme de rapport de fréquences (mots plus fréquents chez Corneille)", color='red', fontsize=fontsize_rectangle, ha='center', va='center')

    #display_vertical_line(plt, index_mean, title='Mean', color='purple')
    #display_vertical_line(plt, index_mode, title='Mode', color='red')
    #display_vertical_line(plt, index_Q1, title='Q1', color='green')
    #display_vertical_line(plt, index_Q2, title='Median', color='green')
    #display_vertical_line(plt, index_Q3, title='Q3', color='green')
    #plt.legend()
    plt.show()

def get_random_extract(author_dataset) -> Tuple[str,str]:
    name = random.choice(list(author_dataset.keys()))
    idx =random.randint(0, len(author_dataset[name])-1)
    comment = f'Extrait {idx+1}/{len(author_dataset[name])} de {name}'
    return (comment, random.choice(author_dataset[name]) )


## PRIVE: Chargement des données et création d'un fichier de statistiques

In [ ]:

# chargement des dataset complets associés à Molière et Corneille
moliere_full_dataset = load_all_books(os.path.join(directory, 'moliere'))
corneille_full_dataset = load_all_books(os.path.join(directory, 'corneille'))
print(f'\nMolière Dataset: {paragraph_count(moliere_full_dataset)} paragraphes ({word_count(moliere_full_dataset)} mots) venant de {len(moliere_full_dataset)} oeuvres:\n{list(moliere_full_dataset.keys())}')
print(f'\nCorneille Dataset: {paragraph_count(corneille_full_dataset)} paragraphes ({word_count(corneille_full_dataset)} mots) venant de {len(corneille_full_dataset)} oeuvres:\n{list(corneille_full_dataset.keys())}')


# split des données entre train et validation
moliere_train_dataset,moliere_validation_dataset,corneille_train_dataset,corneille_validation_dataset = split_train_validation_all_authors(moliere_full_dataset, corneille_full_dataset, percentage_in_train)
print(f'\nMolière Train Dataset: {paragraph_count(moliere_train_dataset)} paragraphes ({word_count(moliere_train_dataset)} mots) venant de {len(moliere_train_dataset)} oeuvres:\n{list(moliere_train_dataset.keys())}')
print(f'\nMolière Validation Dataset: {paragraph_count(moliere_validation_dataset)} paragraphes ({word_count(moliere_validation_dataset)} mots) venant de {len(moliere_validation_dataset)} oeuvres:\n{list(moliere_validation_dataset.keys())}')
print(f'\nCorneille Train Dataset: {paragraph_count(corneille_train_dataset)} paragraphes ({word_count(corneille_train_dataset)} mots) venant de {len(corneille_train_dataset)} oeuvres:\n{list(corneille_train_dataset.keys())}')
print(f'\nCorneille Validation Dataset: {paragraph_count(corneille_validation_dataset)} paragraphes ({word_count(corneille_validation_dataset)} mots) venant de {len(corneille_validation_dataset)} oeuvres:\n{list(corneille_validation_dataset.keys())}')

    

stats_moliere_train_dataset = compute_normalized_words_to_stats_without_stop_words(all_paragraphs(moliere_train_dataset))
moliere_train_dataset_total_word_count = sum([c[0] for c in stats_moliere_train_dataset.values()])

stats_corneille_train_dataset = compute_normalized_words_to_stats_without_stop_words(all_paragraphs(corneille_full_dataset))
corneille_train_dataset_total_word_count = sum([c[0] for c in stats_corneille_train_dataset.values()])

normalized_words = list((set(stats_moliere_train_dataset.keys())|set(stats_corneille_train_dataset.keys())))
normalized_words.sort()
moliere_frequency = []
moliere_count = []
corneille_frequency = []
corneille_count = []


for normalized_word in normalized_words:
    if normalized_word in stats_moliere_train_dataset:
        stat = stats_moliere_train_dataset[normalized_word]
        moliere_frequency.append(stat[0]/moliere_train_dataset_total_word_count)
        moliere_count.append(stat[0])
    else:
        moliere_frequency.append(0)
        moliere_count.append(0)
    if normalized_word in stats_corneille_train_dataset:
        stat = stats_corneille_train_dataset[normalized_word]
        corneille_frequency.append(stat[0]/corneille_train_dataset_total_word_count)
        corneille_count.append(stat[0])
    else:
        corneille_frequency.append(0)
        corneille_count.append(0)

df = pd.DataFrame(
    {'normalized_words': normalized_words,
    'moliere_frequency': moliere_frequency,
    'moliere_count': moliere_count,
    'corneille_frequency' : corneille_frequency,
    'corneille_count' : corneille_count,
    })

# on sauvegarde ces stats sur le disque
df.to_csv(os.path.join(directory, 'stylometrie_stats.csv'), index=False, encoding='utf-8-sig')


def most_common_unnormalized_word(normalized_word: str) -> str:
    res = dict()
    if normalized_word in stats_moliere_train_dataset:
        res = dict(stats_moliere_train_dataset[normalized_word][1])
    if normalized_word in stats_corneille_train_dataset:
        for w,count in stats_corneille_train_dataset[normalized_word][1].items():
            if w in res:
                res[w] += count
            else:
                res[w] = count
    if len(res) == 0:
        return ''
    return max(res, key=res.get)

df_top_words = df.copy()
df_top_words['alpha'] = df_top_words.apply(lambda row: row['moliere_frequency'] / max(row['corneille_frequency'], 1e-9), axis=1)
df_top_words['total_frequency'] = df_top_words['moliere_frequency']+df_top_words['corneille_frequency']
df_top_words = df_top_words.sort_values(by='total_frequency', ascending=False)
df_top_words = df_top_words.drop(columns=['total_frequency'])
df_top_words = df_top_words.head(most_common_normalized_words_count)
df_top_words = df_top_words.sort_values(by='alpha', ascending=False)
df_top_words['unnormalized_word'] = df_top_words['normalized_words'].apply(most_common_unnormalized_word)

df_top_words.to_csv(os.path.join(directory, 'stylometrie_stats_top_words.csv'), index=False, encoding='utf-8-sig')



most_used_words = list(df_top_words['normalized_words'])
most_used_unnormalized_word = list(df_top_words['unnormalized_word'])




# PRIVE: outil de recherche de textes très spécifiques à Molière ou Corneille

In [ ]:
total_words_moliere = sum([c[0] for c in stats_moliere_train_dataset.values()])
total_words_corneille = sum([c[0] for c in stats_corneille_train_dataset.values()])


def compute_word_score_moliere_vs_corneille(count_moliere:int, count_corneille:int):
    if count_moliere+count_corneille<30:
        return 0
    if count_moliere == 0:
        return -1
    if count_corneille == 0:
        return 1
    percentage_moliere = count_moliere/total_words_moliere
    percentage_corneille = count_corneille/total_words_corneille
    if percentage_moliere>2*percentage_corneille:
        return 1
    if percentage_corneille>2*percentage_moliere:
        return -1
    return 0

def to_string_percentage(count:int, total_count:int):
    if total_count<=0:
        return "0%"
    percentage = count/total_count
    return f"{round(100*percentage,3)}%"

def find_most_distinctive_lines(is_moliere: bool) -> None:
    min_score_moliere = 0
    max_score_moliere = 0
    set_most_used_words = set(most_used_words)
    
    author_dataset = moliere_full_dataset  if is_moliere else corneille_full_dataset
    
    for book_name, paragraphs in author_dataset.items():
        for paragraph in paragraphs:
            for line in paragraph.splitlines():
                words = set(split_text(line))
                if len(words)<5:
                    continue
                line_score_moliere_vs_corneille = 0
                comment_moliere = ""
                comment_corneille = ""
                for original_word in words:
                    normalized_word = compute_normalized_word(original_word)
                    if normalized_word not in set_most_used_words:
                        continue
                    count_moliere = stats_moliere_train_dataset[normalized_word][0] if normalized_word in stats_moliere_train_dataset else 0
                    count_corneille = stats_corneille_train_dataset[normalized_word][0] if normalized_word in stats_corneille_train_dataset else 0
                    word_score_moliere_vs_corneille = compute_word_score_moliere_vs_corneille(count_moliere, count_corneille)
                    if word_score_moliere_vs_corneille == 0:
                        continue
                    percent_moliere = to_string_percentage(count_moliere, total_words_moliere)
                    percent_corneille = to_string_percentage(count_corneille, total_words_corneille)
                    if is_moliere:
                        comment = f"{original_word} ({percent_moliere} vs {percent_corneille}) "
                    else:
                        comment = f"{original_word} ({percent_corneille} vs {percent_moliere}) "
                    if word_score_moliere_vs_corneille>0:
                        comment_moliere += comment
                    else:
                        comment_corneille += comment
                    line_score_moliere_vs_corneille += word_score_moliere_vs_corneille
                if is_moliere and comment_corneille:
                    continue
                if not is_moliere and comment_moliere:
                    continue
                if abs(line_score_moliere_vs_corneille)>=2 and (len(comment_moliere)==0 or len(comment_corneille)==0):
                #if total_score_moliere<min_score_moliere or line_score_moliere_vs_corneille>max_score_moliere:
                    min_score_moliere = min(min_score_moliere,line_score_moliere_vs_corneille)
                    max_score_moliere = max(max_score_moliere,line_score_moliere_vs_corneille)
                    print('-'*50)
                    print(f"Oeuvre de {'Molière' if is_moliere else 'Corneille'}: {book_name}")
                    print(line)
                    print(f'Score: {line_score_moliere_vs_corneille}')
                    if comment_moliere:
                        print(f'avantage Molière: {comment_moliere}')
                    if comment_corneille:
                        print(f'avantage Corneille: {comment_corneille}')
                    print('-'*50)
                    print()

print('-'*50)
print('Recherche de lignes spécifiques à Molière')
find_most_distinctive_lines(True)
print()
print('-'*50)
print('Recherche de lignes spécifiques à Corneille')
#find_most_distinctive_lines(False)
print()


#  Mise en situation:
## On veut identifier les auteurs des 2 phrases suivantes:
### - Ah ! ah ! vous voilà. Je suis ravi de vous trouver, Monsieur le coquin.
### - Tes rares qualités te font d'un autre sang

## On montre les fréquences d'apparition de certains mots chez Molière et Corneille 

In [ ]:
def create_table_with_occurences_corneille_moliere(original_words: List[str]) -> pd.DataFrame:

    original_words.sort()
    occurences_moliere = []
    occurences_corneille = []
    frequence_moliere = []
    frequence_corneille = []
    for original_word in original_words:
        normalized_word = compute_normalized_word(original_word)
        occurences_moliere.append(stats_moliere_train_dataset[normalized_word][0] if normalized_word in stats_moliere_train_dataset else 0)
        occurences_corneille.append(stats_corneille_train_dataset[normalized_word][0] if normalized_word in stats_corneille_train_dataset else 0)
        frequence_moliere.append(stats_moliere_train_dataset[normalized_word][0]/total_words_moliere if normalized_word in stats_moliere_train_dataset else 0)
        frequence_corneille.append(stats_corneille_train_dataset[normalized_word][0]/total_words_corneille if normalized_word in stats_corneille_train_dataset else 0)
    df =pd.DataFrame(
        {'Mot': original_words,
        'Occurences Molière': occurences_moliere,
        'Occurences Corneille': occurences_corneille,
        'Fréquence Molière': frequence_moliere,
        'Fréquence Corneille': frequence_corneille} 
        )
    df.set_index(['Mot'],inplace=True)    
    return df   



mots = ['ah', 'monsieur', 'voila', 'tes', 'rares', 'sang']

df = create_table_with_occurences_corneille_moliere(mots)

def format_percentage(value):
    return f'{round(100*value,3)}%'
# Exemple de fonction toto
def create_comment(percent_moliere, percent_corneille):
    if percent_corneille<=0:
        return "Mot présent uniquement chez Molière"
    if percent_moliere<=0:
        return "Mot présent uniquement chez Corneille"
    if percent_moliere>percent_corneille:
        return f"Mot {int(round(percent_moliere/percent_corneille,0))} fois plus utilisé chez Molière"
    else:
        return f"Mot {int(round(percent_corneille/percent_moliere,0))} fois plus utilisé chez Corneille"

# Création de la colonne C en appliquant la méthode toto
df['Commentaire'] = df.apply(lambda row: create_comment(row['Fréquence Molière'], row['Fréquence Corneille']), axis=1)
df['Fréquence chez Molière'] = df['Fréquence Molière'].apply(format_percentage)
df['Fréquence chez Corneille'] = df['Fréquence Corneille'].apply(format_percentage)
df[['Fréquence chez Molière', 'Fréquence chez Corneille', 'Commentaire']]



## En se basant sur le tableau d'occurences ci dessus, qui de Molière ou Corneille a probalement écrit cette ligne:
### "Ah ! ah ! vous voilà. Je suis ravi de vous trouver, Monsieur le coquin."

In [ ]:
# remplacer le "XXX" ci dessous par "Molière" ou par "Corneille"
auteur = "XXX"

## En se basant sur le tableau d'occurences ci dessus, qui de Molière ou Corneille a probalement écrit cette ligne:
### "Tes rares qualités te font d'un autre sang."

In [ ]:
# remplacer le "XXX" ci dessous par "Molière" ou par "Corneille"
auteur = "XXX"

# On affiche le diagramme de rapport de fréquences de toutes les oeuvres de Molière

In [ ]:
for name, extracts in moliere_full_dataset.items():
    affiche_rapport_de_frequences(' '.join(extracts), "oeuvre de Molière : "+name, display_in_percentage = False, display_left_rectangle=True, display_right_rectangle=False)

# On affiche le diagramme de rapport de fréquences de toutes les oeuvres de Corneille

In [ ]:
for name, extracts in corneille_full_dataset.items():
    affiche_rapport_de_frequences(' '.join(extracts), "oeuvre de Corneille : "+name, display_in_percentage = False, display_left_rectangle=False, display_right_rectangle=True)

<hr style="height:2px; border-width:0; color:black; background-color:black">
<span style="font-size: 48px;">1ère partie: </span><span style="font-size: 36px;">(basée sur 1 caractéristique)</span><br><br>
<span style="font-size: 36px;">Déterminer si un texte a été écrit par Molière ou par Corneille, en se basant sur le % de mots dans un intervalle à gauche du diagramme de rapport de fréquences.</span>
<hr style="height:2px; border-width:0; color:black; background-color:black">
<span style="font-size: 24px;"><u>Méthode utilisée:</u></span><br>
<span style="font-size: 24px;">1. On recherche les 100 mots les plus différenciants entre Molière et Corneille</span><br>
<span style="font-size: 24px;">2. On trie ces 100 mot de celui qui est très fréquent chez Molière (et rare chez Corneille), à celui qui est très fréquent chez Corneille (et rare chez Molière)</span><br>
<span style="font-size: 24px;">3. Pour un texte donné, on extrait les mots de ce texte figurant dans les 100 mots les plus diférenciants, et on les place dans un diagramme de rapport de fréquences</span><br>
<span style="font-size: 24px;">4. On calcule le pourcentage de mots dans un intervalle à gauche du diagramme de rapport de fréquences (donc parmi les mots plus fréquents chez Molière que chez Corneille)</span><br>
<span style="font-size: 24px;">5. Si cette valeur est supérieure à un seuil, on attribue le texte à Molière, sinon à Corneille.</span><br>


### On affiche à l'étudiant la valeur de l'erreur pour différentes valeurs du seuil associé au % de mots dans l'intervalle [0, 49] du diagramme de rapport de fréquences 

In [ ]:
index_left = 0
index_right  = 49

affiche_erreur_1_caracteristique(index_left, index_right)

## On propose à l'étudiant de choisir la valeur de la caractéristique

In [ ]:
# la caractéristique à améliorer
# ce '50' sera modifié par l'étudiant
seuil_1ere_caracteristique = 50
validation_erreur = calcul_validation_erreur_moliere_vs_corneille_1_caracteristique(index_left, index_right, seuil_1ere_caracteristique)
print(f'Erreur(Molière ou Corneille?)= {round(100*validation_erreur,1)}%')


### On affiche à l'étudiant la valeur de l'erreur pour différentes valeurs du seuil associé au % de mots dans l'intervalle [0, 19] du diagramme de rapport de fréquences

In [ ]:
index_left = 0
index_right  = 19
affiche_erreur_1_caracteristique(index_left, index_right)

## On propose à l'étudiant de choisir la valeur de la caractéristique

In [ ]:

# la caractéristique à améliorer
# ce '50' sera modifié par l'étudiant
seuil_1ere_caracteristique = 50
validation_erreur = calcul_validation_erreur_moliere_vs_corneille_1_caracteristique(index_left, index_right, seuil_1ere_caracteristique)
print(f'Erreur(Molière ou Corneille?)= {round(100*validation_erreur,1)}%')


# Utilisation de 2 caractéristiques:
## % de mots dans un intervalle à gauche du diagramme de rapport de fréquences Molière/Corneille et
## % de mots dans un autre intervalle à droite du diagramme

In [ ]:

def valeur_2_caracteristiques(text:str, index_left_caracteristique1:int, index_right_caracteristique1:int, index_left_caracteristique2:int, index_right_caracteristique2:int) -> Tuple[float,float]:
    values = compute_words_count_in_text(text)
    valeur_caracteristique1 =  100*sum(values[index_left_caracteristique1: index_right_caracteristique1+1])/max(1,sum(values))
    valeur_caracteristique2 =  100*sum(values[index_left_caracteristique2: index_right_caracteristique2+1])/max(1,sum(values))
    return valeur_caracteristique1,valeur_caracteristique2


def display_2D(text_moliere: List[str], text_corneille: List[str], index_left_caracteristique1:int, index_right_caracteristique1:int, index_left_caracteristique2:int, index_right_caracteristique2:int, seuil_pente:float = None, seuil_ordonnee_a_l_origine:float = None) -> None:
    x_moliere = []
    y_moliere = []
    for t in text_moliere:
        valeur_caracteristique1,valeur_caracteristique2 = valeur_2_caracteristiques(t, index_left_caracteristique1, index_right_caracteristique1, index_left_caracteristique2, index_right_caracteristique2)
        x_moliere.append(valeur_caracteristique2)
        y_moliere.append(valeur_caracteristique1)
    x_corneille = []
    y_corneille = []
    for t in text_corneille:
        valeur_caracteristique1,valeur_caracteristique2 = valeur_2_caracteristiques(t, index_left_caracteristique1, index_right_caracteristique1, index_left_caracteristique2, index_right_caracteristique2)
        x_corneille.append(valeur_caracteristique2)
        y_corneille.append(valeur_caracteristique1)

    # Dessiner les points
    plt.figure(figsize=(8, 8))
    size = 10
    plt.scatter(x_moliere, y_moliere, color='green', label='Oeuvre de Molière', s=size)
    plt.scatter(x_corneille, y_corneille, color='red', label='Oeuvre de Corneille', s=size)

    # Ajouter des détails au graphique
    plt.title('')
    plt.xlabel(f"Valeur de la 2ème caratéristique\n% de mots dans l'intervalle [{index_left_caracteristique2},{index_right_caracteristique2}]", fontsize=12)
    plt.ylabel(f"Valeur de la 1ère caratéristique\n% de mots dans l'intervalle [{index_left_caracteristique1},{index_right_caracteristique1}]", fontsize=12)
    
    x_lim = max(max(x_moliere),max(x_corneille))
    y_lim = max(max(y_moliere),max(y_corneille))
    
    
    if seuil_pente and seuil_ordonnee_a_l_origine:
        # Draw the line using a point and the slope
        plt.axline((0, seuil_ordonnee_a_l_origine), slope=seuil_pente, color='blue', label=f'f(x) = {seuil_pente}x + {seuil_ordonnee_a_l_origine}')
    
    plt.xlim(0, max(x_lim,y_lim))
    plt.ylim(0, max(x_lim,y_lim))
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.legend()
    plt.show()


def is_moliere_2_caracteristiques(t : str, index_left_caracteristique1:int, index_right_caracteristique1:int, index_left_caracteristique2:int, index_right_caracteristique2:int, seuil_pente:float, seuil_ordonnee_a_l_origine:float):
    valeur_caracteristique1,valeur_caracteristique2 = valeur_2_caracteristiques(t, index_left_caracteristique1, index_right_caracteristique1, index_left_caracteristique2, index_right_caracteristique2)
    y = seuil_pente * valeur_caracteristique2 + seuil_ordonnee_a_l_origine
    return valeur_caracteristique1>y


def calcul_confusion_matrix_2_caracteristiques(text_moliere: List[str], text_corneille: List[str], index_left_caracteristique1:int, index_right_caracteristique1:int, index_left_caracteristique2:int, index_right_caracteristique2:int, seuil_pente:float, seuil_ordonne_a_l_oirigne:float) ->Tuple[int,int,int,int]:
    TP = 0 # y_true = Molière ,  y_pred = Molière
    TN = 0 # y_true = Corneille, y_pred = Corneille
    FN = 0 # y_true = Molière,   y_pred = Corneille
    FP = 0 # y_true = Corneille, y_pred = Molière 
    for t in text_moliere:
        if is_moliere_2_caracteristiques(t, index_left_caracteristique1, index_right_caracteristique1, index_left_caracteristique2, index_right_caracteristique2, seuil_pente, seuil_ordonnee_a_l_origine):
            TP += 1
        else:
            FN += 1
    for t in text_corneille:
        if is_moliere_2_caracteristiques(t, index_left_caracteristique1, index_right_caracteristique1, index_left_caracteristique2, index_right_caracteristique2, seuil_pente, seuil_ordonnee_a_l_origine):
            FP += 1
        else:
            TN += 1
    return (TP,TN,FP,FN)
        
    
def calcul_validation_erreur_moliere_vs_corneille_2_caracteristiques(index_left_caracteristique1:int, index_right_caracteristique1:int, index_left_caracteristique2:int, index_right_caracteristique2:int, seuil_pente:float, seuil_ordonnee_a_l_origine:float) -> float:
    return calcul_erreur(*calcul_confusion_matrix_2_caracteristiques(all_paragraphs(moliere_validation_dataset), all_paragraphs(corneille_validation_dataset), index_left_caracteristique1, index_right_caracteristique1, index_left_caracteristique2, index_right_caracteristique2, seuil_pente, seuil_ordonnee_a_l_origine))




# Intervalle associé à la 1ère caractéristique.
# La valeur de la 1ère caractéristique sera le % de mot dans cet intervalle
left_index_caracteristique1 = 0
right_index_caracteristique1 = 12

# Intervalle associé à la 2ème caractéristique.
# La valeur de la 2ème caractéristique sera le % de mot dans cet intervalle
left_index_caracteristique2 = 89
right_index_caracteristique2 =99

# droite permettant de séparer le plan en 2: 
# - les points au dessus de cette droite seront attribués à Molière, les autres à Corneille
seuil_pente = 0.6
seuil_ordonnee_a_l_origine = 5.5


display_2D(all_paragraphs(moliere_validation_dataset), all_paragraphs(corneille_validation_dataset), left_index_caracteristique1,right_index_caracteristique1,left_index_caracteristique2,right_index_caracteristique2, seuil_pente=seuil_pente, seuil_ordonnee_a_l_origine=seuil_ordonnee_a_l_origine)
validation_erreur = calcul_validation_erreur_moliere_vs_corneille_2_caracteristiques(left_index_caracteristique1,right_index_caracteristique1,left_index_caracteristique2,right_index_caracteristique2, seuil_pente=seuil_pente, seuil_ordonnee_a_l_origine=seuil_ordonnee_a_l_origine)
    
print(f'Erreur(Molière ou Corneille?)= {round(100*validation_erreur,2)}%')


# Hyperparameters Search (disabled by default)


In [ ]:
'''

import time 
text = f'seuil_pente,seuil_ordonnee_a_l_origine,left_index_caracteristique1,right_index_caracteristique1,left_index_caracteristique2,right_index_caracteristique2,total_points_caracteristique1,total_points_caracteristique2,erreur\n'
filename = 'c:/temp/hpo_'+str(int(time.time()))+'.csv'

for left_index_caracteristique1 in [0]:
    for right_index_caracteristique2 in [most_common_normalized_words_count-1]:
        for total_points_caracteristique1 in range(10,30+1,1):
            for total_points_caracteristique2 in range(10,30+1,1):
                for seuil_pente in np.arange(0.6, 1.0, 0.05):
                    for seuil_ordonnee_a_l_origine in np.arange(0, 8, 0.5):
                        right_index_caracteristique1 = left_index_caracteristique1+total_points_caracteristique1-1
                        left_index_caracteristique2 = right_index_caracteristique2-total_points_caracteristique2+1
                        validation_erreur = calcul_validation_erreur_moliere_vs_corneille_2_caracteristiques(left_index_caracteristique1,right_index_caracteristique1,left_index_caracteristique2,right_index_caracteristique2, seuil_pente=seuil_pente, seuil_ordonnee_a_l_origine=seuil_ordonnee_a_l_origine)
                        text += f"{seuil_pente},{seuil_ordonnee_a_l_origine},{left_index_caracteristique1},{right_index_caracteristique1},{left_index_caracteristique2},{right_index_caracteristique2},{total_points_caracteristique1},{total_points_caracteristique2},{validation_erreur}\n"
                        with open(filename, 'a') as f:
                            f.write(text)
                        text = ''

'''